## Start environment

In [1]:
%%bash
minikube delete -p dc1
minikube start -p dc1
kubectl config use-context dc1
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update
rm -rf /tmp/tfe
mkdir /tmp/tfe

* Eliminando "dc1" en docker...
* Eliminando /Users/jose/.minikube/machines/dc1...
* Removed all traces of the "dc1" cluster.
* [dc1] minikube v1.30.1 en Darwin 14.6.1 (arm64)
* minikube 1.34.0 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.34.0
* To disable this notice, run: 'minikube config set WantUpdateNotification false'

* Controlador docker seleccionado automáticamente
* Using Docker Desktop driver with root privileges
* Starting control plane node dc1 in cluster dc1
* Pulling base image ...
* Creando docker container (CPUs=2, Memory=4000MB) ...
* Preparando Kubernetes v1.26.3 en Docker 23.0.2...
  - Generando certificados y llaves
  - Iniciando plano de control
  - Configurando reglas RBAC...
* Configurando CNI bridge CNI ...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
* Verifying Kubernetes components...
* Complementos habilitados: storage-provisioner, default-storageclass



! /opt/homebrew/bin/kubectl is version 1.28.2, which may have incompatibilities with Kubernetes 1.26.3.


  - Want kubectl v1.26.3? Try 'minikube kubectl -- get pods -A'
* Done! kubectl is now configured to use "dc1" cluster and "default" namespace by default
Switched to context "dc1".
"hashicorp" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "vault" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "jaegertracing" chart repository
...Successfully got an update from the "jaeger-all-in-one" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "external-secrets" chart repository
...Successfully got an update from the "signoz" chart repository
...Successfully got an update from the "gitlab" ch

## Install PostgresDB

In [2]:
! kubectl create ns tfe

namespace/tfe created


In [3]:
%%bash
# Add Helm repository by Bitnami
helm repo add bitnami https://charts.bitnami.com/bitnami

# Update Helm index charts
helm repo update
kubectl apply -f local-pv.yaml
kubectl apply -f pv-claim.yaml
# volumePermissions for EKS
helm install postgresql-dev -f postgres.yaml bitnami/postgresql -n tfe --set volumePermissions.enabled=true

"bitnami" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "jaeger-all-in-one" chart repository
...Successfully got an update from the "signoz" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "vault" chart repository
...Successfully got an update from the "jaegertracing" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "external-secrets" chart repository
...Successfully got an update from the "gitlab" chart repository
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈Happy Helming!⎈
persistentvolume/postgresql-data created


Error from server (NotFound): error when creating "pv-claim.yaml": namespaces "vault" not found


NAME: postgresql-dev
LAST DEPLOYED: Mon Oct 14 09:56:48 2024
NAMESPACE: tfe
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
CHART NAME: postgresql
CHART VERSION: 16.0.1
APP VERSION: 17.0.0

** Please be patient while the chart is being deployed **

PostgreSQL can be accessed via port 5432 on the following DNS names from within your cluster:

    postgresql-dev.tfe.svc.cluster.local - Read/Write connection

To get the password for "postgres" run:

    export POSTGRES_ADMIN_PASSWORD=$(kubectl get secret --namespace tfe postgresql-dev -o jsonpath="{.data.postgres-password}" | base64 -d)

To get the password for "app1" run:

    export POSTGRES_PASSWORD=$(kubectl get secret --namespace tfe postgresql-dev -o jsonpath="{.data.password}" | base64 -d)

To connect to your database run the following command:

    kubectl run postgresql-dev-client --rm --tty -i --restart='Never' --namespace tfe --image docker.io/bitnami/postgresql:17.0.0-debian-12-r1 --env="PGPASSWORD=$POSTGRES_PASSWORD" \
 

In [ ]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster1_fqdn_8200)
export VAULT_TOKEN=hvs.gw2jGbxgnvR7PjDRwDi9yf21

vault policy write dr-secondary-promotion - <<EOF
path "sys/replication/dr/secondary/promote" {
  capabilities = [ "update" ]
}

# To update the primary to connect
path "sys/replication/dr/secondary/update-primary" {
    capabilities = [ "update" ]
}

# Only if using integrated storage (raft) as the storage backend
# To read the current autopilot status
path "sys/storage/raft/autopilot/state" {
    capabilities = [ "update" , "read" ]
}

path "sys/storage/raft/*" {
    capabilities = [ "update" , "read", "create", "delete", "patch", "sudo" ]
}
EOF

vault write auth/token/roles/failover-handler \
    allowed_policies=dr-secondary-promotion \
    orphan=true \
    renewable=false \
    token_type=batch


vault token create -role=failover-handler -ttl=8h

vault read sys/storage/raft/autopilot/state

In [ ]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster2_fqdn_8200)
export VAULT_TOKEN=<>

vault read sys/storage/raft/autopilot/state